<a href="https://colab.research.google.com/github/duybluemind1988/Data-science/blob/master/Sonion/Compare_all_bigdata_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.datarevenue.com/en-blog/pandas-vs-dask-vs-vaex-vs-modin-vs-rapids-vs-ray